# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import regex as re
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'omw-1.4'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [11]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('disaster_response', con=engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    # replace url
    text = re.sub('http[^ ]+', 'urlplaceholder', text)

    # uncapitalize
    text = text.lower()

    # remove punctuation
    text = re.sub(r'\p{P}+', ' ', text)

    # remove extra space
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    # tokenize
    token_list = word_tokenize(text)

    # remove stopwords
    sw = stopwords.words('english')
    token_list = [w for w in token_list if w not in sw]

    # lemmatize
    lemmatizer = WordNetLemmatizer()
    token_list = [lemmatizer.lemmatize(w) for w in token_list]

    return token_list

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=2022)

pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x12f313280>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
Y_pred = pipeline.predict(X_test)

In [16]:
for i in range(Y_test.shape[1]):
    y_true = Y_test.iloc[:, i]
    y_pred = Y_pred[:, i]
    print('-'*50)
    print(Y_test.columns[i])
    print(classification_report(y_true, y_pred))

--------------------------------------------------
related
              precision    recall  f1-score   support

         0.0       0.71      0.45      0.55      1233
         1.0       0.84      0.94      0.89      3977
         2.0       0.36      0.41      0.38        34

    accuracy                           0.82      5244
   macro avg       0.64      0.60      0.61      5244
weighted avg       0.81      0.82      0.81      5244

--------------------------------------------------
request
              precision    recall  f1-score   support

         0.0       0.90      0.98      0.94      4362
         1.0       0.83      0.48      0.61       882

    accuracy                           0.90      5244
   macro avg       0.86      0.73      0.77      5244
weighted avg       0.89      0.90      0.88      5244

--------------------------------------------------
offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5219
        

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
    'clf__estimator__max_depth': [3, 5, 10],
    'clf__estimator__min_samples_split': [2,5]
}

cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x12f313280>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_depth': [3, 5, 10],
                         'clf__estimator__min_samples_split': [2, 5]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
Y_pred = cv.predict(X_test)

for i in range(Y_test.shape[1]):
    y_true = Y_test.iloc[:, i]
    y_pred = Y_pred[:, i]
    print('-' * 50)
    print(Y_test.columns[i])
    print(classification_report(y_true, y_pred))

--------------------------------------------------
related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1233
         1.0       0.76      1.00      0.86      3977
         2.0       0.00      0.00      0.00        34

    accuracy                           0.76      5244
   macro avg       0.25      0.33      0.29      5244
weighted avg       0.58      0.76      0.65      5244

--------------------------------------------------
request
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      4362
         1.0       0.00      0.00      0.00       882

    accuracy                           0.83      5244
   macro avg       0.42      0.50      0.45      5244
weighted avg       0.69      0.83      0.76      5244

--------------------------------------------------
offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      5219
        

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [23]:
import pickle
pickle.dump(cv.best_estimator_, open('./models/model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.